In [ ]:
import tensorflow as tf

In [ ]:
# Podłączenie kamery - wymaga zainstalowanego OpenCV3
# nie robi nic poza pobraniem obrazu z kamery
# przygotowanie do rozpoznawania twarzy
# MOŻNA POMINĄĆ !

import cv2
import IPython
import time
from io import BytesIO
import PIL

cam = cv2.VideoCapture(0)

def get_frame(cam):
    # Przechwyć obraz
    ret, frame = cam.read()

    # Mirror
    frame = cv2.flip(frame, 1)

    return frame

def array_to_image(a, fmt='png'):
    #Create binary stream object
    f = BytesIO()

    #Convert array to binary stream object
    PIL.Image.fromarray(a).save(f, fmt)

    return IPython.display.Image(data=f.getvalue())

frame = get_frame(cam)
if (frame != None):
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    im = array_to_image(frame)
    d = IPython.display.display("", display_id=1)
    d.update(im)

cam.release()

In [ ]:
from tensorflow.keras.datasets.fashion_mnist import load_data

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
%matplotlib inline

In [ ]:
(X_train, y_train), (X_test, y_test) = load_data()

In [ ]:
X_train.shape

In [ ]:
y_train.shape

In [ ]:
X_train[0]

In [ ]:
plt.figure()
plt.imshow(X_train[0], cmap='Greys')
plt.colorbar()
plt.grid(False)
plt.show()

In [ ]:
y_train[0]

In [ ]:
plt.imshow(X_train[10], cmap='Greys')
plt.colorbar()
plt.grid(False)
plt.show()

In [ ]:
y_train[10]

In [ ]:
# https://github.com/zalandoresearch/fashion-mnist

mapa_kategorii = {
    0 : 'T-shirt/top',
    1 : 'Trouser',
    2 : 'Pullover',
    3 : 'Dress',
    4 : 'Coat',
    5 : 'Sandal',
    6 : 'Shirt',
    7 : 'Sneaker',
    8 : 'Bag',
    9 : 'Ankle boot'
}

In [ ]:
mapa_kategorii[y_train[10]]

In [ ]:
X_train = X_train/255.0

In [ ]:
y_train = y_train.astype(np.int32) 

In [ ]:
model = tf.keras.Sequential([
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(10, activation='softmax')
])

model.compile(
    optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(), 
    metrics=['accuracy']
)

In [ ]:
model.fit(X_train, y_train, epochs=2)

In [ ]:
loss, accuracy = model.evaluate(X_train, y_train, verbose=2)
print("Loss :", loss)
print("Accuracy :", accuracy)

In [ ]:
X_test = X_test/255.0

In [ ]:
y_test = y_test.astype(np.int32)

In [ ]:
y_pred = model.predict(X_test, steps = 10)
print(y_pred.shape)

In [ ]:
y_pred[0]

In [ ]:
np.argmax(y_pred[0])

In [ ]:
mapa_kategorii[np.argmax(y_pred[0])]

In [ ]:
mapa_kategorii[y_test[0]]

In [ ]:
plt.imshow(X_test[0], cmap='Greys')
plt.colorbar()
plt.grid(False)
plt.show()

In [ ]:
plt.grid(False)
plt.xticks(range(10))
plt.yticks([])
plt.bar(range(10), y_pred[0])
plt.ylim([0, 1])

In [ ]:
loss, accuracy = model.evaluate(X_test, y_test, verbose=2)
print("Loss :", loss)
print("Accuracy :", accuracy)

In [ ]:
nr_kategorii = 8
mapa_aktywacji = []
for elem in model.weights[0].numpy():
    mapa_aktywacji.append(elem[nr_kategorii])
    
plt.imshow(np.array(mapa_aktywacji).reshape((28, 28)))

In [ ]:
print("\nDodałem inwersję kolorów ładowanych obrazów\nimg = tf.bitwise.invert(img,name=None)\n")
print("Dzieki temu zaczyna coś rozpoznawać :)")
def decode_img(filepath):
    img = tf.io.read_file(filepath)
    img = tf.image.decode_jpeg(img, channels=1)
    # bez poniższego praktycznie zawsze wykryje torbę
    img = tf.bitwise.invert(img,name=None)
    img = tf.image.convert_image_dtype(img, tf.float32)
    return tf.image.resize(img, [28, 28])

In [ ]:
# tu trzeba wskazać lokalizację plików graficznych 
lista_plikow = tf.data.Dataset.list_files(str('d:/obrazy/*'))

In [ ]:
moj_test = lista_plikow.map(decode_img)

In [ ]:
obraz = None
for elem in moj_test.take(1):
    obraz = elem.numpy()
plt.imshow(obraz.reshape(28, 28), cmap='Greys')
#plt.imshow(obraz.reshape(28, 28), cmap='Greys_r')
plt.colorbar()
plt.show()

In [ ]:
y_pred_moj = model.predict(obraz.reshape(1, 784))

In [ ]:
y_pred_moj

In [ ]:
mapa_kategorii[np.argmax(y_pred_moj)]

In [ ]:
plt.grid(False)
plt.xticks(range(10))
plt.yticks([])
plt.bar(range(10), y_pred_moj[0])
plt.ylim([0, 1])

Dodatek:

Przykładowa prosta sieć konwolucyjna.

In [ ]:
model2 = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), padding='same', activation='relu', input_shape=(28, 28, 1)),
    tf.keras.layers.MaxPooling2D((3, 3)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(10, activation='softmax')
])

model2.compile(
    optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(), 
    metrics=['accuracy']
)


In [ ]:
model2.fit(X_train.reshape(60000, 28, 28, 1), y_train, epochs=2)

In [ ]:
for weight in model2.weights:
    print(weight.shape)